# How to use multimodal prompts

Here we demonstrate how to use prompt templates to format [multimodal](/docs/concepts/multimodality/) inputs to models. 

To use prompt templates in the context of multimodal data, we can templatize elements of the corresponding content block.
For example, below we define a prompt that takes a URL for an image as a parameter:

In [1]:
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate(
    [
        {
            "role": "system",
            "content": "Describe the image provided.",
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source_type": "url",
                    # highlight-next-line
                    "url": "{image_url}",
                },
            ],
        },
    ]
)

Let's use this prompt to pass an image to a [chat model](/docs/concepts/chat_models/#multimodality):

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("anthropic:claude-3-5-sonnet-latest")

url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

chain = prompt | llm
response = chain.invoke({"image_url": url})
print(response.text())

This is a beautiful landscape photograph featuring a wooden boardwalk cutting through a lush green wetland or grassland area. The boardwalk extends straight ahead toward the horizon, creating a strong leading line in the composition. On either side, tall green grasses sway in what appears to be a marsh or prairie environment. The sky is particularly striking, with wispy cirrus clouds streaked across a vibrant blue background. In the distance, there's a tree line that creates the horizon, and the late day lighting gives the scene a warm, peaceful quality. The wooden planks of the boardwalk appear weathered and natural, blending harmoniously with the surrounding environment. It's the kind of scene you might find in a nature preserve or wildlife refuge, designed to allow visitors to experience the natural habitat while protecting the delicate ecosystem.


Note that we can templatize arbitrary elements of the content block:

In [4]:
prompt = ChatPromptTemplate(
    [
        {
            "role": "system",
            "content": "Describe the image provided.",
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source_type": "base64",
                    "mime_type": "{image_mime_type}",
                    "data": "{image_data}",
                    "metadata": {"cache_control": {"type": "{cache_type}"}},
                },
            ],
        },
    ]
)

In [6]:
import base64

import httpx

image_data = base64.b64encode(httpx.get(url).content).decode("utf-8")

chain = prompt | llm
response = chain.invoke(
    {
        "image_data": image_data,
        "image_mime_type": "image/jpeg",
        "cache_type": "ephemeral",
    }
)
print(response.text())

This image shows a beautiful wooden boardwalk cutting through a lush wetland or meadow area. The boardwalk extends straight ahead toward the horizon, creating a strong leading line through the composition. On both sides, tall green grasses sway in what appears to be a marsh or prairie environment. The sky is particularly striking, with wispy cirrus clouds streaked across a bright blue background. In the distance, you can see a tree line along the horizon, and the late day lighting gives the scene a warm, peaceful quality. The wooden planks of the boardwalk appear well-maintained and provide safe passage through what would otherwise be difficult terrain to traverse. It's the kind of scene you might find in a nature preserve or wildlife refuge, designed to allow visitors to experience the natural environment while protecting the delicate ecosystem.
